## Step1. Minio 사용을 위한 Secret과 ServiceAccount 생성

In [1]:
!kubectl apply -f ./yaml/secret.yaml

secret/minio-secret unchanged
serviceaccount/kserve-sa created


In [2]:
!kubectl get sa/kserve-sa -o yaml

apiVersion: v1
kind: ServiceAccount
metadata:
  annotations:
    kubectl.kubernetes.io/last-applied-configuration: |
      {"apiVersion":"v1","kind":"ServiceAccount","metadata":{"annotations":{},"name":"kserve-sa","namespace":"pjt-lecture"},"secrets":[{"name":"minio-secret"}]}
  creationTimestamp: "2023-03-13T16:50:25Z"
  name: kserve-sa
  namespace: pjt-lecture
  resourceVersion: "42093581"
  uid: 51a431c7-6773-4e10-aff1-d76613db9b37
secrets:
- name: minio-secret
- name: kserve-sa-token-qdhkj


In [3]:
!kubectl get secret/minio-secret  -o yaml

apiVersion: v1
data:
  AWS_ACCESS_KEY_ID: bWluaW8=
  AWS_SECRET_ACCESS_KEY: bWluaW8xMjM=
kind: Secret
metadata:
  annotations:
    kubectl.kubernetes.io/last-applied-configuration: |
      {"apiVersion":"v1","data":{"AWS_ACCESS_KEY_ID":"bWluaW8=","AWS_SECRET_ACCESS_KEY":"bWluaW8xMjM="},"kind":"Secret","metadata":{"annotations":{"serving.kserve.io/s3-endpoint":"minio-service.kubeflow:9000","serving.kserve.io/s3-region":"us-east-1","serving.kserve.io/s3-useanoncredential":"false","serving.kserve.io/s3-usehttps":"0","serving.kserve.io/s3-verifyssl":"0"},"name":"minio-secret","namespace":"pjt-lecture"}}
    serving.kserve.io/s3-endpoint: minio-service.kubeflow:9000
    serving.kserve.io/s3-region: us-east-1
    serving.kserve.io/s3-useanoncredential: "false"
    serving.kserve.io/s3-usehttps: "0"
    serving.kserve.io/s3-verifyssl: "0"
  creationTimestamp: "2023-03-08T18:13:29Z"
  name: minio-secret
  namespace: pjt-lecture
  resourceVersion: "30575655"
  uid: 9de7d7ef-3488-4356-bae5-c1181

## Step2. inference service yaml 수정
저장된 모델 파일을 사용하도록 storage uri를 수정해주세요.

In [8]:
!cat ./yaml/inferenceservice.yaml

apiVersion: serving.kserve.io/v1beta1
kind: InferenceService
metadata:
  name: covid-19                          # KFServing InferenceService 명
spec:
  predictor:
    serviceAccountName: kserve-sa         # s3:// 레파지토리(Minio)를 사용하기 위해 이 Kserve InferenceService가 사용할 serviceAccount (Step1.에서 생성함)
    tensorflow:
      storageUri: 's3://{project-namespace}/covid-19/'  # 모델을 저장한 경로 s3://<bucket명>/<저장경로>
      resources:
        limits:
          cpu: '1'
          memory: 2Gi
        requests:
          cpu: 500m
          memory: 1Gi

In [4]:
!kubectl apply -f ./yaml/inferenceservice.yaml

inferenceservice.serving.kserve.io/covid-19 configured


In [6]:
!kubectl get inferenceservice

NAME       URL                                                READY   PREV   LATEST   PREVROLLEDOUTREVISION   LATESTREADYREVISION                AGE
covid-19   http://covid-19.pjt-lecture.sds.swa-kubeflow.com   True           100                              covid-19-predictor-default-00004   4d22h


## Step3. Prediction Test
Endpoint url을 KServe Model UI에서 확인하고, prediction api를 호출합니다.

In [7]:
!MODEL_NAME=covid-19; TEST_JSON="./image_data.json"; \
curl http://covid-19.pjt-lecture.svc.cluster.local/v1/models/covid-19:predict -d @${TEST_JSON}

{
    "predictions": [[0.11127194, 0.160472363, 0.728255749]
    ]
}